In [14]:
import os
import xarray as xr
import numpy as np

USER = os.environ['USER']
os.environ['CESMDATAROOT'] = f'/glade/scratch/{USER}/inputdata'
import pop_tools 

import discrete_obs

In [10]:
ds = xr.open_dataset('/glade/p/cgd/oce/projects/cesm2-marbl/xpersist_cache/3d_fields/Fe.nc')
ds['REGION_MASK'] = pop_tools.get_grid('POP_gx1v7')['REGION_MASK']
ds

<xarray.Dataset>
Dimensions:      (nlat: 384, nlon: 320, z_t: 60)
Coordinates:
  * z_t          (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLONG        (nlat, nlon) float64 ...
    TLAT         (nlat, nlon) float64 ...
    ULONG        (nlat, nlon) float64 ...
    ULAT         (nlat, nlon) float64 ...
Dimensions without coordinates: nlat, nlon
Data variables:
    Fe           (z_t, nlat, nlon) float32 ...
    REGION_MASK  (nlat, nlon) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
Attributes:
    revision:                $Id: tavg.F90 89644 2018-08-04 14:26:01Z klindsay $
    history:                 none
    model_doi_url:           https://doi.org/10.5065/D67H1H0V
    cell_methods:            cell_methods = time: mean ==> the variable value...
    intake_esm_varname:      Fe
    Conventions:             CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf...
    contents:                Diagnostic and Prognostic Variables
    title:                   b.e21.BHIST.f09_g17.CMIP6-historical.011
    source:                  CCSM POP2, the CCSM Ocean Component
    calendar:                All years have exactly  365 days.
    time_period_freq:        month_1
    intake_esm_dataset_key:  ocn.historical.pop.h

In [38]:
import pandas as pd
J = np.arange(50, 201, 1)
I = np.ones(len(J)).astype(np.int) * 200
K = np.random.normal(loc=30, scale=10, size=(len(J))).astype(np.int)
K[K < 0] = 0
K[K > 59] = 59

lines = []
for i, j, k in zip(I, J, K):
    lines.append(
        dict(
            lon=ds.TLONG.values[j, i],
            lat=ds.TLAT.values[j, i],
            depth=ds.z_t.values[k] * 1e-2, 
            REGION_MASK_TEST=ds.REGION_MASK.values[j, i],
            Fe_TEST=ds.Fe.values[k, j, i],
        ))
df = pd.DataFrame(lines).dropna() 
df.to_csv('discrete_obs/dfe-test.csv')
df

,lon,lat,depth,REGION_MASK_TEST,Fe_TEST
0,185.562505,-52.509348,1968.944219,1,0.000613
1,185.562505,-51.975114,115.000000,1,0.000237
2,185.562505,-51.440880,482.736719,1,0.000349
3,185.562505,-50.906646,443.377695,1,0.000345
4,185.562505,-50.372411,175.479043,1,0.000269
...,...,...,...,...,...
146,185.562641,2.546556,251.370156,2,0.000804
147,185.562688,2.815616,984.705859,2,0.000632
148,185.562744,3.084871,285.483652,2,0.000726
149,185.562812,3.354322,408.784648,2,0.000565


In [44]:
df = discrete_obs.open_datastream('test')
df.obs_stream.add_model_field(ds.Fe)
df.obs_stream.add_model_field(ds.REGION_MASK, method='nearest')
df

,Unnamed: 0,lon,lat,depth,REGION_MASK_TEST,Fe_TEST,Fe,REGION_MASK
0,0,185.562505,-52.509348,1968.944219,1,0.000613,0.000613,1.0
1,1,185.562505,-51.975114,115.000000,1,0.000237,0.000237,1.0
2,2,185.562505,-51.440880,482.736719,1,0.000349,0.000349,1.0
3,3,185.562505,-50.906646,443.377695,1,0.000345,0.000345,1.0
4,4,185.562505,-50.372411,175.479043,1,0.000269,0.000269,1.0
...,...,...,...,...,...,...,...,...
146,146,185.562641,2.546556,251.370156,2,0.000804,0.000804,2.0
147,147,185.562688,2.815616,984.705859,2,0.000632,0.000632,2.0
148,148,185.562744,3.084871,285.483652,2,0.000726,0.000726,2.0
149,149,185.562812,3.354322,408.784648,2,0.000565,0.000565,2.0


In [48]:
actual, desired = df.Fe.values, df.Fe_TEST.values
keep = ~(np.isnan(actual) | np.isnan(desired))
np.testing.assert_allclose(actual[keep], desired[keep])

actual, desired = df.REGION_MASK.values, df.REGION_MASK_TEST.values
keep = ~(np.isnan(actual) | np.isnan(desired))
np.testing.assert_allclose(actual[keep], desired[keep])